# Homework 1
(c) 2017 Justin Bois. This work is licensed under a [Creative Commons Attribution License CC-BY 4.0](https://creativecommons.org/licenses/by/4.0/). All code contained herein is licensed under an [MIT license](https://opensource.org/licenses/MIT).

In [4]:
import numpy as np
import pandas as pd

import bokeh.io
import bokeh.plotting

from bokeh.models import Legend
from bokeh.plotting import figure, show, output_file

bokeh.io.output_notebook()

Loading BokehJS ...

## Problem 1.3 (Microtubule catastrophes)
For this exercise, we will analyze the paper by Gardner, Zanic, et al., Depolymerizing kinesins Kip3 and MCAK shape cellular microtubule architecture by differential control of catastrophe, *Cell*, 147, 1092-1103, 2011
<br>
<br>
In this paper, the authors investigated the dynamics of microtubule catastrophe, the switching of a microtubule from a growing to a shrinking state. In particular, they were interested in the time between the start of growth of a microtubule and the catastrophe event. They monitored microtubules by using tubulin (the monomer that comprises a microtubule) that was labeled with a fluorescent marker. As a control to make sure that fluorescent labels and exposure to laser light did not affect the microtubule dynamics, they performed a similar experiment using differential interference contrast (DIC) microscopy. They measured the time until catastrophe with labeled and unlabeled tubulin.
<br>
<br>
We will use their data to generate the plot which is similar to the Fig. 2a of their paper.
<br>
<br>
First, let's load the data into a Pandas `DataFrame`

In [5]:
# Read the data from the data file into a DataFrame
df = pd.read_csv('data/gardner_et_al_2011_time_to_catastrophe_dic.csv', comment='#')

# Let's take a look
df

,time to catastrophe with labeled tubulin (s),time to catastrophe with unlabeled tubulin (s)
0,470,355.0
1,1415,425.0
2,130,540.0
3,280,265.0
4,550,1815.0
5,65,160.0
6,330,370.0
7,325,460.0
8,340,190.0
9,95,130.0


The data above are not tidy. Let's remember the three rules for the tidy data:
* Each variable forms a column.
* Each observation forms a separate row.
* Each type of observational unit forms a separate table.

To tidy these data, we should drop `NaN`s. We could not use these values for analysis anyways, as we need to match control measurements (unlabeled) with the measurements of interest (labeled)

In [6]:
# Drop NaN values
df = df.dropna()

# Let's look at it
df.head()

,time to catastrophe with labeled tubulin (s),time to catastrophe with unlabeled tubulin (s)
0,470,355.0
1,1415,425.0
2,130,540.0
3,280,265.0
4,550,1815.0


Apparently, the `NaN` datapoints have been removed.
<br>
<br>
In the Fig. 2a of their paper, Gardner, Zanic, et al. have the empirical cumulative distribution function (ECDF). We will try to reconstruct this plot. First, we need to write a function `ecdf_vals(data)` which takes a one-dimensional Numpy array (or Pandas `Series`; same construction will work) of data anad returns the `x` and `y` values for plotting the ECDF. The definition of ECDF is
\begin{align}
ECDF(x) = fraction \ of \ data \ points \leq x
\end{align}

In [7]:
def ecdf_vals(data):
    """Function returns the x and y values for the plotting of ECDF.
        Input: data (Numpy array or Pandas Series)
        Output: a pair of Numpy arrays (xaxis data and yaxis data)."""
    x = np.sort(data)
    y = np.arange(1, len(data)+1)/len(data)
    
    return x, y

In [8]:
# First let's rename the columns to have shorter names
rename_dict = {'time to catastrophe with labeled tubulin (s)' : 'tc_lab',
               'time to catastrophe with unlabeled tubulin (s)' : 'tc_unlab'}

df = df.rename(columns=rename_dict)

# Let's look at it
df.head()

,tc_lab,tc_unlab
0,470,355.0
1,1415,425.0
2,130,540.0
3,280,265.0
4,550,1815.0


In [9]:
# Select labeled tubulin data
d_lab = df['tc_lab']

# Select unlabeled tubulin data
d_unlab = df['tc_unlab']

# Apply the ECDF functions to both labeled and unlabeled data
xlab, ylab = ecdf_vals(d_lab)
xunlab, yunlab = ecdf_vals(d_unlab)

# Now use bokeh to the ECDFs on the single graph
# Set up the plot
f = bokeh.plotting.figure(plot_height=300,
                          plot_width=500,
                          x_axis_label='time to catastrophe (s)',
                          y_axis_label='ECDF')

# Add a scatter plot
f1 = f.circle(xlab, ylab, color='red')
f2 = f.circle(xunlab, yunlab, color='green')

# Make a legend object
legend = Legend(items=[
            ('labeled tubulin', [f1]),
            ('unlabeled tubulin', [f2])
            ])
# Add legend
f.add_layout(legend)
f.legend.location = 'bottom_right'

# Add axis labels
f.xaxis.axis_label = 'time to catastrophe (s)'
f.yaxis.axis_label = 'ECDF'



bokeh.io.show(f)

Although the ECDFs are often plotted as the scatter graphs, it is not a typical convention. Given the definition of the ECDF, it is defined for *all* values of *x* along the real *x-axis*. So, formally, the ECDF should be plotted as a line.
<br>
<br>
Now, we will write a function `plot_ecdf_formal(data)` that takes a one-dimensional Numpy array of dat and returns a Bokeh figure with the ECDF plotted as a line.

In [24]:
def plot_ecdf_formal(data, data_legend='data', plot_height=300, plot_width=500, 
                     x_axis_label='data', y_axis_label='ECDF'):
    """Returns the figure with the line plot of the ECDF data."""
    # Compute the ECDF
    x, y = ecdf_vals(data)
    
    # Make the Bokeh figure
    f = bokeh.plotting.figure(plot_height=plot_height,
                              plot_width=plot_width,
                              x_axis_label=x_axis_label,
                              y_axis_label=y_axis_label)
    
    # Make a line plot
    f1 = f.line(x,y, color='red')

    # Make a legend object
    legend = Legend(items=[
            (data_legend, [f1])
            ])
    # Add legend
    f.add_layout(legend)
    f.legend.location = 'bottom_right'

    # Add axis labels
    f.xaxis.axis_label = x_axis_label
    f.yaxis.axis_label = y_axis_label

    bokeh.io.show(f)
    
    return None

Let's try to reproduce our ECDF plot for the data for labeled tubulin.

In [25]:
plot_ecdf_formal(d_lab, data_legend='labeled tubulin', 
                 x_axis_label='time to catastrophe (s)')

The plot looks good!

## Bayes's Theorem and statistical inference
Let's do a quick recap of the *Bayes's theorem* and apply it to the data before we will talk about the marginalization. We start from the product rule, where we consider three events, $A$, $B$, and $C$ and their probabilities of occuring. Product rule states that
\begin{align}
P(A, B \mid C) = P(A \mid B,C) \, P(B \mid C) \quad \textbf{(product rule)}
\end{align}
This says that the probability of $A$ *and* $B$ occuring, given that $C$ happened is equal to probability of $A$ occuring given that $B$ and $C$ happened, multiplied by the probability of $B$ occuring, given that $C$ happened.
<br>
<br>
Now we can add some meaning to the events $A$, $B$, and $C$. How about this:
* $ A = H_{i} $ is the hypothesis (or parameter value) that we are testing.
* $ B = D $ is the measured data.
* $ C = I $ is all the other information we know. 
<br>
Now we can rewrite the product rule as
\begin{align}
P(H_{i}, D \mid I) = P(H_{i} \mid D,I) \, P(D \mid I)
\end{align}
The *and* operation is commutative. It means that $P(H_{i}, D \mid I) = P(D, H_{i} \mid I)$. It means that we can express these equations as 
\begin{align}
P(H_{i} \mid D,I) \, P(D \mid I) = P(D \mid H_{i},I) \, P(H_{i} \mid I)
\end{align}
After rearranging, we get
\begin{align}
P(H_{i} \mid D,I) = \frac{P(D \mid H_{i},I) \, P(H_{i} \mid I)}{P(D \mid I)} 
\quad \textbf{(Bayes's theorem)}
\end{align}
This is exactly what we wanted - all the quantities on the right side have meaning and they describe the probability that our hypothesis is true, given the data and the other information we have. Let's assign the names to the terms in the equation and rewrite the equation as follows:
\begin{align}
posterior = \frac{likelihood \times prior}{evidence}
\end{align}

*Note: The Bayes's Theorem is a statement about the probability, which is valid for both Bayesian and frequentist interpretation of the probability.*
<br>
<br>
Let's go through each term: <br>
* **The prior probability.** This represents the plausibility of the hypothesis $H_{i}$ given everything we know *before* we did the experiment to get the data. <br>
<br>
* **The likelihood.** The likelihood describes how likely it is to obtain the data, *given that the hypothesis $H_{i}$ is true.* It also contains information about our expectations from data, given our measurement method. For instance, instrument noise, its model, and the other external circumstances.
<br>
<br>
* **The evidence.** It can be computed from the likelihood and prior and is also called the *marginal likelihood*.
<br>
<br>
* **The posterior probability.** And this is what we want - how plausible is the hypothesis, given that we have measured some new data? It is calculated directly from the likelihood and prior.

## Marginalization
We said that the evidence can be computed from the likelihood and the prior. To see this, let's apply the sum rule to the posterior probability:
<br>
\begin{align}
P(H_{j} \mid D,I) + P(\bar{H_{j}} \mid D, I) = 1 \\[1em]
P(H_{j} \mid D,I) + \sum_{i \neq j}{P(H_{i} \mid D,I)} = 1 \\
\sum_{i}{P(H_{i} \mid D,I)} = 1
\end{align}
for some hypothesis $H_{j}$. We can now apply the Bayes's Theorem as 
<br>
\begin{align}
\sum_{i}{P(H_{i} \mid D,I)} = \sum_{i}{\frac{P(D \mid H_{i}, I) \, P(H_{i} \mid I)}{P(D \mid I)}} = 1 \\[1em]
\sum_{i}{P(H_{i} \mid D,I)} = \frac{1}{{P(D \mid I)}} \sum_{i}{P(D \mid H_{i}, I) \, P(H_{i} \mid I)} = 1 \\[1em]
\end{align}
Therefore, we can compute the *evidence* by summing over the products of likelihoods and priors:
<br>
\begin{align}
P(D \mid I) = \sum_{i}{P(D \mid H_{i}, I) \, P(H_{i} \mid I)} \\[1em]
\end{align}
This process of elimination of a variable (in this case the hypotheses) from a probability by summing is called *marginalization*. <br>
How to grasp the marginalization? I think that we can imagine that we are trying to look through the plausible "Hypothesis space"  on all the possible hypotheses, taking into account the measured data and the prior information to assess the "influence" and the implications of each hypothesis for the observed experimental data, given the prior information.

## Probability distribution of a marginalized parameter
Let's say we have a statistical model with two continuous parameters, that is, $\theta = (\theta_{1}, \theta{2})$. A statement of Bayes's theorem in this case is
\begin{align}
P(\theta \mid D,I) = \frac{P(D \mid \theta,I) \, P(\theta \mid I)}{P(D \mid I)}
\end{align}
This can be explicitly written as 
\begin{align}
P(\theta_{1}, \theta_{2} \mid D,I) = \frac{P(D \mid \theta_{1}, \theta_{2},I) \, P(\theta_{1}, \theta_{2} \mid I)}{P(D \mid I)}
\end{align}
Let's try to marginalize the statistical model by $\theta_{2}$ to obtain $P(\theta_{1} \mid D,I)$:
Considering that we have continuous hypothesis space, we can write
\begin{align}
P(\theta_{1} \mid D,I) = \int{ \frac{P(D \mid \theta_{1}, \theta_{2},I) \, P(\theta_{1}, \theta_{2} \mid I)}{P(D \mid I)} d\theta_{2}}
\end{align}

